In [2]:
import numpy as np
import scipy as sp
import math

In [158]:
def fun(i, j, k):
    return np.sin(i + j + k)

def hilbert_3(i, j, k, m, p):
    return 1 / (1 + i + j + k + m + p)

def hilbert_5(i, j, k, m, p):
    return 1 / (1 + i + j + k + m + p)

def random_3(i, j, k):
    return 1 / (np.random.normal(1, 0.001, (i.shape)) + i + j + k)

def random_5(i, j, k):
    return 1 / (np.random.normal(1, 0.001, (i.shape)) + i + j + k)

In [159]:
# control by rank
def trsvd(matrix, rank):
    U, S, Vh = np.linalg.svd(matrix, full_matrices=True)
    return U[:,0:rank], S[0:rank], Vh[0:rank, :]

#control by an error
def trsvd_ec(matrix, error):
    U, S, Vh = np.linalg.svd(matrix, full_matrices=True)
    sigmas_revrece_cumsum = np.cumsum(S[::-1])[::-1]
    rank = len(sigmas_revrece_cumsum[sigmas_revrece_cumsum > error])
    print(f"Approximant error: {sigmas_revrece_cumsum[rank] if rank < S.shape[0] else 0}")
    return U[:,0:rank], S[0:rank], Vh[0:rank, :]


# With ranks
def stHOSVD(A, ranks):
    shape = A.shape
    u = []
    scan_matrix_1 = np.reshape(A, (shape[0], int(A.size / shape[0]),))
    matrix_for_cycle = scan_matrix_1
    ranks_for_cycle = [shape[i] for i in range(1, len(shape))]
    
    for i in range(len(shape)):
        #print(matrix_for_cycle.shape)
        U, S, Vh = trsvd(matrix_for_cycle, ranks[i])
        u.append(U)
        # print(Vh)
        # print(np.diag(S) @ Vh)
        matrix_for_cycle = np.reshape(np.transpose(np.diag(S) @ Vh, (1, 0)), (ranks_for_cycle[0], int(Vh.size / ranks_for_cycle[0])))
        ranks_for_cycle = ranks_for_cycle[1::]  + [Vh.shape[0]]
        if i == (len(shape) - 1):
            return np.reshape(matrix_for_cycle, ranks), u

# With error
def stHOSVD_ec(A, error):
    shape = A.shape
    u = []
    ranks = []
    scan_matrix_1 = np.reshape(A, (shape[0], int(A.size / shape[0]),))
    matrix_for_cycle = scan_matrix_1
    ranks_for_cycle = [shape[i] for i in range(1, len(shape))]
    
    for i in range(len(shape)):
        # print(matrix_for_cycle.shape)
        U, S, Vh = trsvd_ec(matrix_for_cycle, error / np.sqrt(len(shape)))
        u.append(U)
        ranks.append(Vh.shape[0])
        # print(Vh)
        # print(np.diag(S) @ Vh)
        matrix_for_cycle = np.reshape(np.transpose(np.diag(S) @ Vh, (1, 0)), (ranks_for_cycle[0], int(Vh.size / ranks_for_cycle[0])))
        ranks_for_cycle = ranks_for_cycle[1::]  + [Vh.shape[0]]
        if i == (len(shape) - 1):
            return np.reshape(matrix_for_cycle, ranks), u


In [160]:
fun = random_3
n = 100
tensor = np.fromfunction(fun, (n, n, n), dtype=float)
# sprint(tensor)

In [161]:
ranks = (2, 5, 3)
g1, u1 = stHOSVD(tensor, ranks)

In [162]:
tensor_reconstructed_with_ranks = np.einsum('ia,jb,kc,abc->ijk', *u1, g1)

In [163]:
print(f"Error for TD with ranks {ranks}: {np.linalg.norm(tensor_reconstructed_with_ranks - tensor)}")

Error for TD with ranks (2, 5, 3): 0.7003648832034041


In [168]:
error = 0.001
g2, u2 = stHOSVD_ec(tensor, error)

Approximant error: 0.0005742509269299775
Approximant error: 0.0005728540514833458
Approximant error: 0.0005543696167528775


In [169]:
tensor_reconstructed_with_ec = np.einsum('ia,jb,kc,abc->ijk', *u2, g2)

In [170]:
print(f"Tuker ranks for error {error}: {g2.shape}")
print(np.linalg.norm(tensor_reconstructed_with_ec - tensor))

Tuker ranks for error 0.001: (23, 17, 13)
0.00014497409259120632


In [171]:
# memory consumption
mem = np.sum([v.size for v in u2]) + g2.size
print(f"Initial mem: {tensor.size}")
print(f"Tucker mem: {mem}")
print(f"Relation: {tensor.size / mem}")

Initial mem: 1000000
Tucker mem: 10383
Relation: 96.31127805065974
